In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import unidecode
import string
import random
import time
import math

has_gpu = torch.cuda.is_available()

In [2]:
text = unidecode.unidecode(open('verses.txt').read())[21080:]
print(text[:200])

250

1. Cuckoo Song

SUMER is icumen in,
  Lhude sing cuccu!
Groweth sed, and bloweth med,
  And springth the wude nu--
          Sing cuccu!

Awe bleteth after lomb,
  Lhouth after calve cu;
Bulluc s


In [3]:
all_characters = string.printable
num_characters = len(all_characters)

In [4]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(CharRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=num_layers)
        self.linear = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        x = self.embedding(input.view(-1,1))
        output, hidden = self.gru(x.view(-1, 1, self.hidden_size), hidden)
        x = self.linear(output.view(-1, self.hidden_size))
        return x, hidden
    
    def init_hidden(self):
        return Variable(torch.zeros(self.num_layers, 1, self.hidden_size))

In [5]:
def sentence_to_tensor(sentence):
    output = torch.zeros(len(sentence)).long()
    for i in range(len(sentence)):
        output[i]= all_characters.index(sentence[i])
    return Variable(output)

In [6]:
def random_chunk(chunk_len=200):
    start = random.randint(0, len(text) - chunk_len)
    return text[start : start + chunk_len + 1]

print(random_chunk())

def random_input_output(chunk_len=200):
    random_text = random_chunk(chunk_len=chunk_len)
    inp = sentence_to_tensor(random_text[:-1])
    out = sentence_to_tensor(random_text[1:])
    return inp, out

yet!--and I can tarry
  Your love's protracted growing:
June rear'd that bunch of flowers you carry,
  From seeds of April's sowing.

I plant a heartful now: some seed
  At least is sure to strike,
And


In [7]:
def train(model, optimizer, inp, target):
    hidden = model.init_hidden()
    model.zero_grad()
    loss = 0
    chunk_len = len(inp)
    
    if has_gpu:
        hidden, inp, target = hidden.cuda(), inp.cuda(), target.cuda()
    
    output, _ = model(inp, hidden)
    loss = criterion(output, target)
        
    loss.backward()
    optimizer.step()
    
    return loss.data[0] / chunk_len
    

In [8]:
def generate_text(model, prime_string = 'A', predict_len=200, temperature=0.8):
    hidden = model.init_hidden()
    predicted = prime_string
    prime_input = sentence_to_tensor(prime_string)
    
    if has_gpu:
        hidden, prime_input = hidden.cuda(), prime_input.cuda()
    
    for c in range(len(prime_string)-1):
        _, hidden = model(prime_input[c], hidden)
    
    inp = prime_input[-1]
        
    for i in range(predict_len):
        output, hidden = model(inp, hidden)
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = sentence_to_tensor(predicted_char).cuda()
        
    return predicted


In [9]:
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

n_epochs = 5000
print_every = 100
plot_every = 10
hidden_size = 100
num_layers = 1
lr = 0.005

model = CharRNN(num_characters, hidden_size, num_characters, num_layers)
if has_gpu:
    model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    loss = train(model, optimizer, *random_input_output())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]=====>' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(generate_text(model, 'Be', 200), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

 1s (100 2%) 0.0101]=====>
Beng the of Shor loon
       lom the samt weruders mangl as dout moung eal fouc-rou ciem to sea the thed hern men ker tures ing the the ow hot enminle ass sher wheong cou mand--
    in yeken seadrdin8en 

 1s (200 4%) 0.0113]=====>
Bey wing biams ing for uney to 's bee,
The se ard bise hous forusis son, und the qrovith  cive,
  The the hep I onWigept?
   The Mery;
The in ther seur cry of shous moun, wou bower, leses by ar thouthe  

 2s (300 6%) 0.0107]=====>
Bery shir plath thest I burly this avent wind pigll.


Ep pyest anis
  EHg this she DidlE bouss alls nazen as leving aet mone:
    Ers, is bair hand viss and and mounys brea bus though shout heren
  Whi 

 3s (400 8%) 0.0109]=====>
Ben, nounce
        For mew in wist

She dake corongle, your as as my rumppere dery lis there the neid,
And is my mand your a deang
  ghe coughle her ungide, a'   nith runge,
As dord shand; way.


Whear 

 4s (500 10%) 0.0102]=====>
Be, and gart and
  The wearth my liveere's e

 27s (3600 72%) 0.0083]=====>
Ben, his plechand unde
  The His sprives obout you agod like his parte
  The hair the quit, my Some;
  For the deye we nor cloud, still that seep
      That hath lart the walls on the get,
  Freep his v 

 27s (3700 74%) 0.0091]=====>
Befor me!

Gud with there ain that  one to pell,
  Which the many in vails thought dorcal dalp it,
  And blaze thou He hasly bead in in that who splest.

Gipss, of memony her me the When the hight of of 

 28s (3800 76%) 0.0102]=====>
Beserty creep calt the spee,
Thou like my desting like dead
  Summer from furm share up so mert,
    The in my be and she pan deck
    Deadley god. 1575-159

745 away braideale that do Shall,
    And le 

 29s (3900 78%) 0.0092]=====>
Berand well and towe;
And with the fires,
As pass wire the gam'd was and the to gomeke and gife is waters by treat hered.

And littiness the lear the with rotes love?'
  The shand to the gow and light
, 

 30s (4000 80%) 0.0085]=====>
Bettoretore cad hoose us whent

In [10]:
print(generate_text(model, predict_len=500))

A') thee wandom bright,
  As sif the plower copers full,
Then but make foom seem'd were, the foress of earth;
  I I in them's the with sweeter earber;
Thou could wing in bowred Thou rose's and fale,
A shade the sheed. clent minder.

They faired yet of her glread,
  I sweet bloth who the hold,
  She lips, the peared in the Sone blod.

  The Forth, not this hast here,
    And rest the sad in earther:
So be and the Fletiging mint.
  Sorrow now shought with this her steep penited
  When that speary k
